# In Depth A/B Testing - Lab

## Introduction

In this lab, you'll explore a survey from Kaggle regarding budding data scientists. With this, you'll form some initial hypotheses, and test them using the tools you've acquired to date. 

## Objectives

You will be able to:
* Conduct t-tests and an ANOVA on a real-world dataset and interpret the results

## Load the Dataset and Perform a Brief Exploration

The data is stored in a file called **multipleChoiceResponses_cleaned.csv**. Feel free to check out the original dataset referenced at the bottom of this lab, although this cleaned version will undoubtedly be easier to work with. Additionally, meta-data regarding the questions is stored in a file name **schema.csv**. Load in the data itself as a Pandas DataFrame, and take a moment to briefly get acquainted with it.

> Note: If you can't get the file to load properly, try changing the encoding format as in `encoding='latin1'`

In [232]:
import pandas as pd
import pandasql as ps
import numpy as np
import scipy.stats
from flatiron_stats import welch_df,welch_t,p_value_welch_ttest

df = pd.read_csv('multipleChoiceResponses_cleaned.csv')

df1= df[['FormalEducation','AdjustedCompensation']].copy()


q1 = """
SELECT *
FROM df1
WHERE FormalEducation = "Bachelor's degree"
"""

q2 = """
SELECT *
FROM df1
WHERE FormalEducation = "Master's degree"
"""
q3 = """
SELECT *
FROM df1
WHERE FormalEducation = "Doctoral degree" 
"""
q1adj = """
SELECT *
FROM df1
WHERE FormalEducation = "Bachelor's degree" and 
AdjustedCompensation > 0 and AdjustedCompensation < 1000000
"""

q2adj = """
SELECT *
FROM df1
WHERE FormalEducation = "Master's degree" and
AdjustedCompensation > 0 and AdjustedCompensation < 1000000
"""
q3adj = """
SELECT *
FROM df1
WHERE FormalEducation = "Doctoral degree" and 
AdjustedCompensation > 0 and AdjustedCompensation < 1000000
"""

sortedbac = ps.sqldf(q1)
sortedbac = sortedbac.dropna()
sortedmas = ps.sqldf(q2)
sortedmas = sortedmas.dropna()
sorteddoc = ps.sqldf(q3)
sorteddoc = sorteddoc.dropna()
sortedbacadj = ps.sqldf(q1adj)
sortedbacadj = sortedbacadj.dropna()
sortedmasadj = ps.sqldf(q2adj)
sortedmasadj = sortedmasadj.dropna()
sorteddocadj = ps.sqldf(q3adj)
sorteddocadj = sorteddocadj.dropna()



/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [233]:
a = np.array(sortedbac['AdjustedCompensation']) # a is bachelor
b = np.array(sortedmas['AdjustedCompensation']) # b is master
c = np.array(sorteddoc['AdjustedCompensation']) # c is doctoral
d = np.array(sortedbacadj['AdjustedCompensation']) # a is bachelor adj
e = np.array(sortedmasadj['AdjustedCompensation']) # b is master adj
f = np.array(sorteddocadj['AdjustedCompensation']) # c is doctoral adj
# bmean =  round(sortedbac['AdjustedCompensation'].mean(),2)
# bstd =  round(sortedbac['AdjustedCompensation'].std(),2)
# mmean = round(sortedmas['AdjustedCompensation'].mean(),2)
# mstd
welch_t(d,f)

11.76723361673775

## Wages and Education

You've been asked to determine whether education is impactful to salary. Develop a hypothesis test to compare the salaries of those with Master's degrees to those with Bachelor's degrees. Are the two statistically different according to your results?

> Note: The relevant features are stored in the 'FormalEducation' and 'AdjustedCompensation' features.

You may import the functions stored in the `flatiron_stats.py` file to help perform your hypothesis tests. It contains the stats functions that you previously coded: `welch_t(a,b)`, `welch_df(a, b)`, and `p_value(a, b, two_sided=False)`. 

Note that `scipy.stats.ttest_ind(a, b, equal_var=False)` performs a two-sided Welch's t-test and that p-values derived from two-sided tests are two times the p-values derived from one-sided tests. See the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html) for more information.    

In [234]:
#H0 : The salary is the same for people who have a masters degree as people who have a bachelors degree
#Ha : They are not the same
welch_t(a,b)

0.43786693335411514

In [235]:
welch_df(a,b)

1350.0828973008781

In [236]:
p_value_welch_ttest(a, b, two_sided=False)

0.33077639451272445

## Wages and Education II

Now perform a similar statistical test comparing the AdjustedCompensation of those with Bachelor's degrees and those with Doctorates. If you haven't already, be sure to explore the distribution of the AdjustedCompensation feature for any anomalies. 

In [237]:
import statistics as s
s1 = s.median(list(c))
s2 = s.median(list(a))
len(a)
p_value_welch_ttest(a, c, two_sided=False)
len(a)


1107

In [238]:
# def removeoutliers(array):   
#     mean = np.mean(array)
#     standard_deviation = np.std(array)
#     distance_from_mean = abs(array - mean)
#     max_deviations = 2
#     not_outlier = distance_from_mean < max_deviations * standard_deviation
#     no_outliers = array[not_outlier]
#     return no_outliers
# c_array = removeoutliers(c)
# a_array = removeoutliers(a)


p_value_welch_ttest(d, f, two_sided=False)



0.0

In [224]:


# Median Values: 
# s1:74131.92 
# s2:38399.4
# Sample sizes: 
# s1: 967 
# s2: 1107
# Welch's t-test p-value: 0.1568238199472023


# Repeated Test with Ouliers Removed:
# Sample sizes: 
# s1: 964 
# s2: 1103
# Welch's t-test p-value with outliers removed: 0.0



## Wages and Education III

Remember the multiple comparisons problem; rather than continuing on like this, perform an ANOVA test between the various 'FormalEducation' categories and their relation to 'AdjustedCompensation'.

In [246]:
#Your code here
print(scipy.stats.f_oneway(a,b)) #bachelors vs masters w/ outliers
print(scipy.stats.f_oneway(a,c))# bachelors vs doctorate w/ outliers
print(scipy.stats.f_oneway(d,f)) # bachelors vs doctorate outliers removed
print(scipy.stats.f_oneway(d,e,f)) # all with outliers removed


F_onewayResult(statistic=0.28293500054527615, pvalue=0.5948221120110966)
F_onewayResult(statistic=1.1635336047175568, pvalue=0.2808594595947254)
F_onewayResult(statistic=140.86429977565842, pvalue=1.7960011262560295e-31)
F_onewayResult(statistic=78.85867368470785, pvalue=2.554734506584201e-34)
KruskalResult(statistic=171.64825258611825, pvalue=3.22984436924927e-39)


## Additional Resources

Here's the original source where the data was taken from:  
    [Kaggle Machine Learning & Data Science Survey 2017](https://www.kaggle.com/kaggle/kaggle-survey-2017)

## Summary

In this lab, you practiced conducting actual hypothesis tests on actual data. From this, you saw how dependent results can be on the initial problem formulation, including preprocessing!